### Setup

In [1]:
import polars as pl
from pathlib import Path

In [4]:
FACT = Path("../data/processed/fact_sales.parquet")

In [5]:
# Lazy scan (no full load)
lf = pl.scan_parquet(FACT)
lf

### Dates, uniques, control

In [9]:
# Small helper: count rows quickly
lf.select(pl.len()).collect(engine='auto')

len
u32
58327370


In [10]:
# Date range
lf.select(pl.min("date").alias("min_date"), pl.max("date").alias("max_date")).collect(engine='auto')

min_date,max_date
date,date
2011-01-29,2016-04-24


In [12]:
# Unique series count and check 1 row per (id, date)
n_pairs = lf.select(pl.len().alias("rows"),
                    pl.col("id").n_unique().alias("n_series"),
                    pl.concat_str([pl.col("id"), pl.col("date").cast(pl.Utf8)], separator="|").n_unique().alias("n_id_date_pairs")
).collect(engine='auto')
n_pairs

rows,n_series,n_id_date_pairs
u32,u32,u32
58327370,30490,58327370


In [14]:
# Duplicates
dups = (
    lf.group_by(["id","date"])
      .agg(pl.len().alias("cnt"))
      .filter(pl.col("cnt") > 1)
      .limit(5)
      .collect(engine='auto')
)
dups

id,date,cnt
str,date,u32


### Missingness and joins

In [15]:
# Overall null rate for sell_price
null_rate = (
    lf.select(((pl.col("sell_price").is_null()).cast(pl.Int8).sum() / pl.len()).alias("sell_price_null_rate"))
      .collect(engine='auto')
)
null_rate

sell_price_null_rate
f64
0.210869


In [16]:
# Null rate by year 
by_year = (
    lf.group_by("year")
      .agg(
          pl.len().alias("n"),
          pl.col("sell_price").is_null().sum().alias("nulls")
      )
      .with_columns((pl.col("nulls")/pl.col("n")).alias("null_rate"))
      .sort("year")
      .collect(engine='auto')
)
by_year

year,n,nulls,null_rate
i64,u32,u32,f64
2011,10275130,5167100,0.502874
2012,11159340,3933782,0.35251
2013,11128850,2259520,0.203033
2014,11128850,839570,0.075441
2015,11128850,98928,0.008889
2016,3506350,513,0.000146


### Validation

In [18]:
neg = lf.filter(pl.col("qty") < 0).select(pl.len()).collect(engine='auto')

# Share of zeros (ile zer – ważne w retail)
zero_share = (
    lf.select((pl.col("qty") == 0).cast(pl.Int8).sum().alias("zeros"), pl.len().alias("n"))
      .with_columns((pl.col("zeros")/pl.col("n")).alias("zero_rate"))
      .collect(engine='auto')
)
neg, zero_share

(shape: (1, 1)
 ┌─────┐
 │ len │
 │ --- │
 │ u32 │
 ╞═════╡
 │ 0   │
 └─────┘,
 shape: (1, 3)
 ┌──────────┬──────────┬───────────┐
 │ zeros    ┆ n        ┆ zero_rate │
 │ ---      ┆ ---      ┆ ---       │
 │ i64      ┆ u32      ┆ f64       │
 ╞══════════╪══════════╪═══════════╡
 │ 39777094 ┆ 58327370 ┆ 0.681963  │
 └──────────┴──────────┴───────────┘)

### Seasonality

In [ ]:
# Daily total by weekday (1..7)
weekday_profile = (
    lf.group_by("wday")
      .agg(pl.col("qty").sum().alias("qty_total"))
      .sort("wday")
      .collect(engine='auto')
)
weekday_profile


#TODO: export for viz in pbi

wday,qty_total
i64,i32
1,11383849
2,11269626
3,8968860
4,8290677
5,8192736
6,8245967
7,9343694


In [ ]:

wm_nulls = lf.select(pl.col("wm_yr_wk").is_null().sum().alias("wm_yr_wk_nulls")).collect(engine='auto')
wm_nulls


wm_yr_wk_nulls
u32
0


In [23]:
import os
DEBUG_N_SERIES = int(os.getenv("DEBUG_N_SERIES", "500"))

# Pick top N series by total volume
topN_ids = (
    lf.group_by("id").agg(pl.col("qty").sum().alias("tot"))
      .sort(pl.col("tot"), descending=True)
      .limit(DEBUG_N_SERIES)
      .select("id")
      .collect(engine='auto')
)

ids_list = topN_ids.get_column("id").to_list()

subset = (
    lf.filter(pl.col("id").is_in(ids_list))
      .collect(engine='auto')
)

subset.shape  # sanity


(956500, 21)